In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
import time
from itertools import product, combinations
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tensorflow.python.ops import random_ops
import warnings
from numba import jit
warnings.filterwarnings('ignore')

In [2]:
np.random.seed(12345)
tf.random.set_seed(12345)

In [3]:
class PhysicsInformedNN:
    def __init__(self, x, t, p, n, q, G_L, R_n, R_p, J_n, J_p, layers):
        X = np.concatenate([x, t, p, n], 1)
        self.lb = X.min(0)
        self.ub = X.max(0)
        
        self.X = X
        self.x = X[:, 0:1]
        self.t = X[:, 1:2]
        self.p = X[:, 2:3]
        self.n = X[:, 3:4]
        
        self.G_L = G_L
        self.R_n = R_n
        self.R_p = R_p
        
        self.layers = layers
        self.J_n = J_n
        self.J_p = J_p
        self.q = q
    
        self.weights, self.biases = self.initialize_NN(layers)
        
        self.lambda_1 = tf.Variable([0.0], dtype=tf.float32)
        
        self.G_L = tf.Variable([G_L], dtype=tf.float32)
        self.R_n = tf.Variable([R_n], dtype=tf.float32)
        
        self.sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(allow_soft_placement=True, log_device_placement=True))
        
        self.x_tf = tf.placeholder(tf.float32, shape=[None, self.x.shape[1]])
        self.t_tf = tf.placeholder(tf.float32, shape=[None, self.t.shape[1]])
        self.p_tf = tf.placeholder(tf.float32, shape=[None, self.p.shape[1]])
        self.n_tf = tf.placeholder(tf.float32, shape=[None, self.n.shape[1]])
        self.J_p_tf = tf.placeholder(tf.float32, shape=[None, self.J_p.shape[1]])
        self.J_n_tf = tf.placeholder(tf.float32, shape=[None, self.J_n.shape[1]])
        self.p_pred, self.n_pred, self.J_n_pred, self.J_p_pred, self.f_n_pred, self.f_p_pred = self.net_NS(self.x_tf, self.t_tf,
                                                                                                           self.n_tf, self.p_tf, self.J_n_tf, 
                                                                                                           self.J_p_tf)
        
        self.loss = tf.reduce_sum(tf.square(self.p_tf - self.p_pred)) + \
                    tf.reduce_sum(tf.square(self.n_tf - self.n_pred)) + \
                    tf.reduce_sum(tf.square(self.J_p_tf-self.J_p_pred)) +\
                    tf.reduce_sum(tf.square(self.J_n_tf-self.J_n_pred)) +\
                    tf.reduce_sum(tf.square(self.f_n_pred)) + \
                    tf.reduce_sum(tf.square(self.f_p_pred))
                    
        self.optimizer_Adam = tf.compat.v1.train.AdamOptimizer()
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)
        
        init = tf.compat.v1.global_variables_initializer()
        self.sess.run(init)
        
    def initialize_NN(self, layers):
        weights = []
        biases = []
        num_layers = len(layers)
        for l in range(0, num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1, layers[l+1]]), dtype=tf.float32)  # Corrected shape of bias
            weights.append(W)
            biases.append(b)
        return weights, biases
    
    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.random.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)

    
    def neural_net(self, X, weights, biases):
        num_layers = len(weights) + 1
        H = 2.0 * (X - self.lb) / (self.ub - self.lb) - 1.0
        for l in range(0, num_layers-2):
            W = weights[l]
            b = biases[l]
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        return Y
    
    def net_NS(self, x, t, n, p, J_n_input, J_p_input):
        
        lambda_1 = self.lambda_1
    
        f_pass = self.neural_net(tf.concat([x, t, n, p, J_n_input, J_p_input], 1), self.weights, self.biases)
    
        del_P = f_pass[:, 3:4]
        del_n = f_pass[:, 2:3]
        J_n_output = f_pass[:, 4:5]
        J_p_output = f_pass[:, 5:6]
    
        del_p_t = tf.gradients(del_P, t)[0]
        del_n_t = tf.gradients(del_n, t)[0]
        del_J_n = tf.gradients(J_n_output, x)[0]
        del_J_p = tf.gradients(J_p_output, x)[0]
    
        f_n = del_n_t - lambda_1 * (del_J_n / self.q) + self.R_n - self.G_L
        f_p = del_p_t - lambda_1 * (del_J_p / self.q) + self.R_p - self.G_L
    
        return del_p_t, del_n_t, del_J_n, del_J_p, f_n, f_p

    
    def callback(self, loss, lambda_1):
        print('Loss: %.3e, Lambda 1: %.3f' % (loss, lambda_1))
    @jit
    def train(self, nIter):
        tf_dict = {self.x_tf: self.x, self.t_tf: self.t, self.p_tf: self.p, self.n_tf: self.n, self.J_p_tf: self.J_p, self.J_n_tf: self.J_n}
        
        start_time = time.time()
        for it in range(nIter):
            self.sess.run(self.train_op_Adam, tf_dict)
            
            if it % 10 == 0:
                elapsed = time.time() - start_time
                loss_value = self.sess.run(self.loss, tf_dict)
                lambda_1_value = self.sess.run(self.lambda_1)
                print('It: %d, Loss: %.3e, Lambda 1: %.3f, Time: %.2f' % (it, loss_value, lambda_1_value, elapsed))
                start_time = time.time()
                
    def predict(self, x_star, t_star, p_star, n_star, J_p_star, J_n_star):
        tf_dict = {self.x_tf: x_star, self.t_tf: t_star, self.n_tf: n_star, self.p_tf: p_star, self.J_p_tf: J_p_star, self.J_n_tf: J_n_star}
        
        p_pred = self.sess.run(self.p_pred, tf_dict)
        n_pred = self.sess.run(self.n_pred, tf_dict)
        J_n_pred = self.sess.run(self.J_n_pred, tf_dict)
        J_p_pred = self.sess.run(self.J_p_pred, tf_dict)
        
        return p_pred, n_pred, J_n_pred, J_p_pred



In [4]:
def plot_solution(X_star, u_star, index):
    # Assuming X_star has shape (num_points, 2) where each row is (x, t)
    # Assuming u_star has shape (num_points, 4) where each row is (p, n, J_n, J_p)
    
    lb = X_star.min(0)
    ub = X_star.max(0)
    nn = 200
    x = np.linspace(lb[0], ub[0], nn)
    t = np.linspace(lb[1], ub[1], nn)
    X, T = np.meshgrid(x, t)
    
    # Interpolate the solution values
    U_star_p = griddata(X_star, u_star[:,0], (X, T), method='cubic')
    U_star_n = griddata(X_star, u_star[:,1], (X, T), method='cubic')
    U_star_Jn = griddata(X_star, u_star[:,2], (X, T), method='cubic')
    U_star_Jp = griddata(X_star, u_star[:,3], (X, T), method='cubic')
    
    fig = plt.figure(index)
    
    ax1 = fig.add_subplot(221, projection='3d')
    ax1.plot_surface(X, T, U_star_p, cmap='viridis')
    ax1.set_title('p(x,t)')
    ax1.set_xlabel('x')
    ax1.set_ylabel('t')
    ax1.set_zlabel('p')
    axisEqual3D(ax1)
    
    ax2 = fig.add_subplot(222, projection='3d')
    ax2.plot_surface(X, T, U_star_n, cmap='viridis')
    ax2.set_title('n(x,t)')
    ax2.set_xlabel('x')
    ax2.set_ylabel('t')
    ax2.set_zlabel('n')
    axisEqual3D(ax2)
    
    ax3 = fig.add_subplot(223, projection='3d')
    ax3.plot_surface(X, T, U_star_Jn, cmap='viridis')
    ax3.set_title('J_n(x,t)')
    ax3.set_xlabel('x')
    ax3.set_ylabel('t')
    ax3.set_zlabel('J_n')
    axisEqual3D(ax3)
    
    ax4 = fig.add_subplot(224, projection='3d')
    ax4.plot_surface(X, T, U_star_Jp, cmap='viridis')
    ax4.set_title('J_p(x,t)')
    ax4.set_xlabel('x')
    ax4.set_ylabel('t')
    ax4.set_zlabel('J_p')
    axisEqual3D(ax4)
    
    plt.show()

def axisEqual3D(ax):
    extents = np.array([getattr(ax, 'get_{}lim'.format(dim))() for dim in 'xyz'])
    sz = extents[:,1] - extents[:,0]
    centers = np.mean(extents, axis=1)
    maxsize = max(abs(sz))
    r = maxsize/4
    for ctr, dim in zip(centers, 'xyz'):
        getattr(ax, 'set_{}lim'.format(dim))(ctr - r, ctr + r)
